<a href="https://colab.research.google.com/github/Mariana-G/machine-learning-group/blob/master/Copia_de_test_chicas_muertos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_csv('/content/train.csv')
df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
23557,32708,NaN,Zimbabwe,2020-04-03,9.0,1.0
23558,32709,NaN,Zimbabwe,2020-04-04,9.0,1.0
23559,32710,NaN,Zimbabwe,2020-04-05,9.0,1.0
23560,32711,NaN,Zimbabwe,2020-04-06,10.0,1.0


In [0]:
cases_deaths_ar = df[(df['Country_Region']	== 'Argentina')]
cases_deaths_ar

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
462,643,NaN,Argentina,2020-01-22,0.0,0.0
463,644,NaN,Argentina,2020-01-23,0.0,0.0
464,645,NaN,Argentina,2020-01-24,0.0,0.0
465,646,NaN,Argentina,2020-01-25,0.0,0.0
466,647,NaN,Argentina,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
534,715,NaN,Argentina,2020-04-03,1265.0,39.0
535,716,NaN,Argentina,2020-04-04,1451.0,43.0
536,717,NaN,Argentina,2020-04-05,1451.0,44.0
537,718,NaN,Argentina,2020-04-06,1554.0,48.0


In [0]:
cases_deaths_ar['ConfirmedCases'] = cases_deaths_ar['ConfirmedCases'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
cases_deaths_ar

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
462,643,NaN,Argentina,2020-01-22,0,0.0
463,644,NaN,Argentina,2020-01-23,0,0.0
464,645,NaN,Argentina,2020-01-24,0,0.0
465,646,NaN,Argentina,2020-01-25,0,0.0
466,647,NaN,Argentina,2020-01-26,0,0.0
...,...,...,...,...,...,...
534,715,NaN,Argentina,2020-04-03,1265,39.0
535,716,NaN,Argentina,2020-04-04,1451,43.0
536,717,NaN,Argentina,2020-04-05,1451,44.0
537,718,NaN,Argentina,2020-04-06,1554,48.0


In [0]:
loc_group = ["Province_State", "Country_Region"]

In [0]:
def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
    for col in loc_group:
        df[col].fillna("none", inplace=True)
    return df

In [0]:
df = preprocess(df)
sub_df = preprocess(pd.read_csv("/content/test.csv"))
df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,none,Afghanistan,2020-01-22,0.0,0.0
1,2,none,Afghanistan,2020-01-23,0.0,0.0
2,3,none,Afghanistan,2020-01-24,0.0,0.0
3,4,none,Afghanistan,2020-01-25,0.0,0.0
4,5,none,Afghanistan,2020-01-26,0.0,0.0


In [0]:
df["Date"].min(), df["Date"].max()

(Timestamp('2020-01-22 00:00:00'), Timestamp('2020-04-07 00:00:00'))

In [0]:
TARGETS = ["ConfirmedCases", "Fatalities"]

for col in TARGETS:
    df[col] = np.log1p(df[col])

    print(TARGETS)
  

['ConfirmedCases', 'Fatalities']
['ConfirmedCases', 'Fatalities']


In [0]:
print(df[col])

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
23557    0.693147
23558    0.693147
23559    0.693147
23560    0.693147
23561    1.098612
Name: Fatalities, Length: 23562, dtype: float64


In [0]:
for col in TARGETS:
    df["prev_{}".format(col)] = df.groupby(loc_group)[col].shift()

In [0]:
df = df[df["Date"] > df["Date"].min()].copy()
df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities
1,2,none,Afghanistan,2020-01-23,0.0,0.0,0.0,0.0
2,3,none,Afghanistan,2020-01-24,0.0,0.0,0.0,0.0
3,4,none,Afghanistan,2020-01-25,0.0,0.0,0.0,0.0
4,5,none,Afghanistan,2020-01-26,0.0,0.0,0.0,0.0
5,6,none,Afghanistan,2020-01-27,0.0,0.0,0.0,0.0


In [0]:
from datetime import timedelta

TEST_DAYS = 7

TRAIN_LAST =  - timedelta(days=TEST_DAYS)

TEST_FIRST = sub_df["Date"].min()
TEST_DAYS = (df["Date"].max() - TEST_FIRST).days + 1

dev_df, test_df = df[df["Date"] < TEST_FIRST].copy(), df[df["Date"] >= TEST_FIRST].copy()
dev_df.shape, test_df.shape

((19278, 8), (3978, 8))

In [0]:
print(TRAIN_LAST)

-7 days, 0:00:00


In [0]:
print(TEST_FIRST)

2020-03-26 00:00:00


In [0]:
sub_df["Date"].max()

Timestamp('2020-05-07 00:00:00')

In [0]:
TEST_DAYS

13

In [0]:
print(df["Date"].max())

2020-04-07 00:00:00


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

model = Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=False)),
                  ('linear', LinearRegression())])

features = ["prev_{}".format(col) for col in TARGETS]

model.fit(dev_df[features], dev_df[TARGETS])

[mean_squared_error(dev_df[TARGETS[i]], model.predict(dev_df[features])[:, i]) for i in range(len(TARGETS))]

[0.04280332071852165, 0.01009902181270343]

In [0]:
print(features)

['prev_ConfirmedCases', 'prev_Fatalities']


In [0]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def evaluate(df):
    error = 0
    for col in TARGETS:
        error += rmse(df[col].values, df["pred_{}".format(col)].values)
    return np.round(error/len(TARGETS), 5)


def predict(test_df, first_day, num_days, val=False):

    y_pred = np.clip(model.predict(test_df.loc[test_df["Date"] == first_day][features]), None, 16)

    for i, col in enumerate(TARGETS):
        test_df["pred_{}".format(col)] = 0
        test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i]

    if val:
      print(first_day, evaluate(test_df[test_df["Date"] == first_day]))

    for d in range(1, num_days):
        y_pred = np.clip(model.predict(y_pred), None, 16)
        date = first_day + timedelta(days=d)

        for i, col in enumerate(TARGETS):
            test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

        if val:
            print(date, evaluate(test_df[test_df["Date"] == date]))
        
    return test_df

test_df = predict(test_df, TEST_FIRST, TEST_DAYS, val=True)
evaluate(test_df)

2020-03-26 00:00:00 0.25798
2020-03-27 00:00:00 0.32955
2020-03-28 00:00:00 0.39398
2020-03-29 00:00:00 0.45843
2020-03-30 00:00:00 0.51629
2020-03-31 00:00:00 0.58445
2020-04-01 00:00:00 0.64146
2020-04-02 00:00:00 0.6987
2020-04-03 00:00:00 0.74463
2020-04-04 00:00:00 0.79215
2020-04-05 00:00:00 0.83994
2020-04-06 00:00:00 0.88312
2020-04-07 00:00:00 0.92971


0.65549